# Cargar los eventos almacenados en la base de datos.

In [1]:
import psycopg2

import pandas as pd

In [2]:
conexion = psycopg2.connect(user ='postgres',
                            password = 'sergio',
                            host = 'localhost',
                            port = '5432',
                            database = 'futbol'
                            )

cursor = conexion.cursor()
sql = 'SELECT jornada_id, partido_id, expanded_minute, period, equipo, h_a, score, ht_score, player_name, accion, x, y FROM campograma'
cursor.execute(sql)
filas = cursor.fetchall()
datos = pd.DataFrame(filas)

cursor.close()
conexion.close()

# Tratar los eventos, para quedarse solo con las faltas cometidas y las columnas necesarias para las visualizaciones.

In [3]:
datos = pd.DataFrame(filas, columns = ['jornada_id', 'partido_id', 'minuto', 'periodo', 'equipo', 'estadio', 'resultado', 
                                        'resultado_descanso', 'jugador', 'accion', 'x', 'y'])

df_faltas = datos.loc[(datos['accion'] == 'foulCommitted')|(datos['accion'] == 'foulCommitted|penaltyConceded')].reset_index(drop = True)
df_faltas['id_equipo'] = df_faltas.partido_id.str.cat(df_faltas.equipo, sep='_')
df_faltas[['jornada','competicion','temporada']] = df_faltas.jornada_id.str.split('_',expand = True)
df_faltas['id'] = df_faltas.id_equipo.str.cat(df_faltas.temporada, sep='_')

df_faltas = df_faltas[['id','jornada','competicion','temporada','partido_id','minuto','periodo','equipo','estadio','resultado',
                        'resultado_descanso','jugador','accion','x','y']]

df_faltas

,id,jornada,competicion,temporada,partido_id,minuto,periodo,equipo,estadio,resultado,resultado_descanso,jugador,accion,x,y
0,Osasuna-Sevilla_Osasuna_22-23,1,LaLiga,22-23,Osasuna-Sevilla,1,FirstHalf,Osasuna,h,2 : 1,1 : 1,Juan Cruz,foulCommitted,60.0,93.8
1,Osasuna-Sevilla_Osasuna_22-23,1,LaLiga,22-23,Osasuna-Sevilla,11,FirstHalf,Osasuna,h,2 : 1,1 : 1,Aimar Oroz,foulCommitted,75.6,91.2
2,Osasuna-Sevilla_Osasuna_22-23,1,LaLiga,22-23,Osasuna-Sevilla,17,FirstHalf,Osasuna,h,2 : 1,1 : 1,Juan Cruz,foulCommitted,65.6,87.0
3,Osasuna-Sevilla_Osasuna_22-23,1,LaLiga,22-23,Osasuna-Sevilla,24,FirstHalf,Osasuna,h,2 : 1,1 : 1,Kike Barja,foulCommitted,46.0,4.0
4,Osasuna-Sevilla_Osasuna_22-23,1,LaLiga,22-23,Osasuna-Sevilla,33,FirstHalf,Osasuna,h,2 : 1,1 : 1,David García,foulCommitted,34.8,24.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9762,Rayo Vallecano-Almeria_Almeria_22-23,20,LaLiga,22-23,Rayo Vallecano-Almeria,19,FirstHalf,Almeria,a,2 : 0,0 : 0,Samú Costa,foulCommitted,33.0,44.4
9763,Rayo Vallecano-Almeria_Almeria_22-23,20,LaLiga,22-23,Rayo Vallecano-Almeria,28,FirstHalf,Almeria,a,2 : 0,0 : 0,Samú Costa,foulCommitted,61.5,72.0
9764,Rayo Vallecano-Almeria_Almeria_22-23,20,LaLiga,22-23,Rayo Vallecano-Almeria,30,FirstHalf,Almeria,a,2 : 0,0 : 0,Sergio Akieme,foulCommitted,32.5,87.2
9765,Rayo Vallecano-Almeria_Almeria_22-23,20,LaLiga,22-23,Rayo Vallecano-Almeria,45,FirstHalf,Almeria,a,2 : 0,0 : 0,Lucas Robertone,foulCommitted,32.9,33.5
